In [1]:
from attention_project_utils import *

In [2]:
from suite2p import run_s2p
import imagej

In [3]:
import matplotlib.pyplot as plt
import os
import numpy as np
import scyjava_config

In [ ]:
%matplotlib notebook

## Path to your Fiji installation that includes `ImageJ-win64.exe`

In [4]:
fiji_path = 'fiji'

## Initialize Fiji

In [5]:
scyjava_config.add_options('-Xmx2g')
ij = imagej.init(fiji_path)
ij.getVersion()
ij.ui().showUI()

## Select directory containng `.h5` and `.nd2` data

In [ ]:
fish_dir = select_directory()

In [ ]:
print(fish_dir)
#nd2_files = list(filter(lambda x: "wholebrain" not in x, list_files(fish_dir, extension='nd2')))
nd2_files = list(filter(lambda x: "wholebrain" not in x, list_files(fish_dir, extension='tif')))
h5_files = list_files(fish_dir, extension='h5')

processed_dir = fish_dir + '/processed'
avi_dir = processed_dir + '/avi'
raw_tif_dir = processed_dir + '/raw tif'
combined_tif_dir = processed_dir + '/combined tif/'
intensity_dir = processed_dir + '/intensity'
registered_tif_dir = processed_dir + '/registered tif'
neural_dir = processed_dir + '/neural'
behavior_dir = processed_dir + '/behavior'

for directory in [processed_dir, avi_dir, raw_tif_dir, combined_tif_dir,
                  intensity_dir, registered_tif_dir, neural_dir, behavior_dir]:
    create_directory(directory)

## Macro for converting `.nd2` image data to `.tif` files

In [ ]:
nd2_to_tif = """
#@ String nd2_path
#@ String save_path
setBatchMode(true);
run("Bio-Formats Importer", "open=[" + nd2_path + "] autoscale color_mode=Default rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT");
//run("Rotate 90 Degrees Left");
run("8-bit");
saveAs("Tiff", save_path);
run("Close All");
"""

In [ ]:
#raw_tif_files = list(map(lambda x: x.replace('nd2', 'tif').replace(fish_dir, raw_tif_dir), nd2_files))
raw_tif_files = list(map(lambda x: x.replace('nd2', 'tif').replace(fish_dir, raw_tif_dir), nd2_files))

for nd2_file, tif_file  in zip(nd2_files, raw_tif_files):
    args = {
        'nd2_path': nd2_file,
        'save_path': tif_file
    }
    ij.py.run_script('ijm', nd2_to_tif, args)

## Macro for combining multiple `.tif` files into one `.tif` file

In [ ]:
combine_tif = """
#@ String[] tif_files
#@ String save_directory
#@output String slices
setBatchMode(true);
open(tif_files[0]);
image1 = getTitle();
slices = toString(nSlices);

for (i = 1; i < tif_files.length; ++i) {
    open(tif_files[i]);
    image2 = getTitle();
    slices = slices + " " + toString(nSlices);
    run("Concatenate...", "open image1=" + image1 + " image2="+ image2);
    image1 = getTitle();
}
saveAs("Tiff", save_directory + "combined.tif");
run("Close All");
"""

In [ ]:
args = {
    'tif_files': raw_tif_files,
    'save_directory': combined_tif_dir + '/'
}

slices = ij.py.run_script('ijm', combine_tif, args).getOutput('slices')
slices = list(map(lambda x: int(x), slices.split(" ")))

## Run suite2p

In [ ]:
ops = run_s2p.default_ops()
ops['batch_size'] = 5000
ops['roidetect'] = False
ops['reg_tif'] = True
ops['save_folder'] = processed_dir
ops['reg_tif'] = True

db = {
      'h5py': [],
      'h5py_key': 'data',
      'look_one_level_down': False,
      'data_path': [combined_tif_dir],                     
      'subfolders': [], 'reg_tif': True
    }

In [ ]:
opsEnd=run_s2p.run_s2p(ops=ops,db=db)

## Macro for watershed segmentation

In [ ]:
segmentation = """
#@ String image_path

open(image_path);
window_name = getTitle();
output_directory = replace(image_path, window_name, "");
run("Z Project...", "projection=[Average Intensity]");
selectWindow(window_name);
run("Close");
avg = getTitle();
run("8-bit");
setTool("rectangle");
waitForUser("Pause", "Draw midbox"); // Ask for input ROI
saveAs("XY Coordinates", output_directory + "Midline.txt");
run("Select None");
setTool("polygon");
waitForUser("Pause", "Draw a mask");
run("Create Mask");
saveAs("Tiff", output_directory + "Mask.tif");
imageCalculator("AND create stack", avg , "Mask.tif");
run("Median...", "radius=1 stack");
run("Enhance Contrast...", "saturated=0.3 normalize");
run("Morphological Filters", "operation=[White Top Hat] element=Disk radius=5");
run("Invert");
run("Watershed Segmentation");
waitForUser("Pause", "Check results");
saveAs("Tiff", output_directory + "label.tif");
run("Analyze Regions", "area perimeter circularity centroid ");
saveAs("Results", output_directory + "label-Morphometry.csv");
run("Close All");
"""

In [ ]:
registered_tif = processed_dir + '/plane0/reg_tif/file000_chan0.tif'

args = {
    'image_path': registered_tif,
}
ij.py.run_script('ijm', segmentation, args)

## Macro for separating registered combined `.tif` imaging data

In [ ]:
separate = """
#@ String combined_tif_path
#@ String save_path
#@ String[] tif_names
#@ Integer[] slices
setBatchMode(true);
open(combined_tif_path);
main_window = getTitle();
folder = replace(combined_tif_path, main_window, "");
save_folder = folder + "results\\";

File.makeDirectory(replace(combined_tif_path, main_window, "results"));

for (i = 0; i < slices.length; ++i) {
    if (i < slices.length - 1) {
        run("Make Substack...", "delete slices=1-" + slices[i]);
        curr_win = getTitle();
    }
    run("8-bit");
    saveAs("Tiff", save_path + tif_names[i]);
    run("Close");
}
run("Close All");

"""

In [ ]:
tif_names = list(map(lambda x: x.split('/')[-1], raw_tif_files))

args = {
    'combined_tif_path': processed_dir + '/plane0/reg_tif/file000_chan0.tif',
    'save_path': registered_tif_dir + '/',
    'tif_names': tif_names,
    'slices': slices
}
ij.py.run_script('ijm', separate, args)

## Macro for getting the intensities of ROIs 

In [ ]:
print(registered_tif_dir)
get_intensity = """
setBatchMode(true)
#@ String label
#@ String save_folder
#@ String[] file_list
setBatchMode(true);
for (i = 0; i < file_list.length; ++i) {
    temp = split(file_list[i], "/");
    getInt(file_list[i], label, save_folder + replace(temp[temp.length - 1], ".tif", "/"));
}

function getInt(image, label, save_folder) {
    if (!File.exists(save_folder)) {
        File.makeDirectory(save_folder);
    }
    open(label);
    label_window = replace(getTitle(), ".tif", "");
    open(image);
    image = getTitle();
    run("8-bit");
    for(i = 1; i <= nSlices; ++i) {
        setSlice(i); // start from first frame
        run("Duplicate...", "use"); // isolate the frame
        current_slice = getTitle();
        print(current_slice);
        curr = replace(current_slice, ".tif", "");
        //print(curr);
        run("Intensity Measurements 2D/3D", "input=" + curr + " labels=" + label_window +  " mean");
        saveAs("Results", save_folder + "int_t" + toString(i) + ".csv");
        run("Close");
        selectWindow(current_slice);
        run("Close");
    }
}
"""

In [ ]:
list_files(registered_tif_dir)

label = processed_dir + '/plane0/reg_tif/label.tif'

args = {
    'label': label,
    'save_folder': intensity_dir + '/',
    'file_list': list_files(registered_tif_dir),
}
ij.py.run_script('ijm', get_intensity, args)

## Convert `.h5` files to `.avi` files

In [ ]:
fish_dir = 'E:\\new folder\\002 ANALYSIS\\4v6_4v2\\2P032-A4\\'
h5_files = list_files(fish_dir, extension='h5')
avi_dir = fish_dir


In [ ]:
#main = 'E:\\new folder\\002 ANALYSIS\\compare PC in 2 dots\\' # added
#fish = ['2P021-A6', '2P022-A7', '2P026-A2', '2P027-A2', '2P029-A2', # added
#            '2P032-A4', '2P035-A1', '2P035-A2', '2P035-A3', '2P035-A4',
#           '2P035-A5', '2P042-A3', '2P044-A1']

#for ith in range(12,13):# added
#    fish_dir = main + fish[ith] # added
#processed_dir = fish_dir + '/processed'
#    avi_dir = processed_dir + '/avi' # added
fps = 100
 #   fps = 300 # added
 #   if ith == 10 or ith == 11: # added
 #       fps = 100 # added
 #   for directory in [processed_dir, avi_dir]: # added
 #       create_directory(directory) # added
    
h5_files = list_files(fish_dir, extension='h5')
for hdf_path in h5_files:
    filename, file_extension = os.path.splitext(hdf_path)
    vid = h5py.File(hdf_path, 'r')
    keys = vid.keys()
    images = []
    dotstop = ''
    for i in keys:
        ind = re.findall('\d+', i)
        images.append(int(ind[0]))
    images = sorted(images)
    video_name = filename + dotstop + '.avi'
    height, width = vid[list(keys)[0]][:].shape

    #video = cv2.VideoWriter(os.path.splitext(hdf_path)[0].replace(fish_dir , avi_dir) + '.avi',
                            #cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (width, height), isColor=False)
    
    video = cv2.VideoWriter(os.path.splitext(hdf_path)[0].replace(fish_dir , avi_dir) + '.avi', 0, fps, (width, height), isColor=False)

    for im in tqdm(sorted(keys, key=lambda x: int(x))):
        img = vid[im][:]
        video.write(img)

    cv2.destroyAllWindows()
    video.release()

In [ ]:
print(avi_dir)

## Combine and process `.csv` files

In [ ]:
morphology = pd.read_csv(processed_dir + '/plane0/reg_tif/label-Morphometry.csv')
neurons = np.array(morphology[(morphology.Area < 200) & (morphology.Area > 15) & (morphology.Circularity > 0.5) & (morphology.Circularity < 1.5)].Label)

for trial_dir in list_directories(intensity_dir):
    split = trial_dir.split('/')[-1].replace('-', '_').replace('.', '_').split('_')
    fish_trial = split[1] + '-' + split[-1]
    df = pd.concat([pd.read_csv(int_file).set_index('Label').rename(
                   columns={'Mean': int(int_file.split('t')[-1].split('.')[0]) - 1})
                   for int_file in list_files(trial_dir)], axis=1)

    df = df.reindex(sorted(df.columns), axis=1).transpose()
    for t in df:
        df[t] = (df[t] - df[t].quantile(.25)) / df[t].quantile(.25)
    new_df = df[neurons].transpose()
    new_df.to_csv(neural_dir + '/' + fish_trial + '.csv')

## (Optional) Getting eye angles from `.avi` files using `zfish_track`
Get `zfish_track` from <https://github.com/kclamar/zfish-track>, or

Install with `pip`: `pip install git+git://github.com/kclamar/zfish-track.git`


In [ ]:
!pip install git+git://github.com/kclamar/zfish-track.git

In [ ]:
import zfish_track

In [ ]:
videos = list_files(avi_dir, '.avi')

In [ ]:
cv2.destroyAllWindows()

In [ ]:
for video_path in videos:
    video_name = os.path.split(video_path)[1]
    split = video_name.replace('-', '_').replace('.', '_').split('_')
    fish_trial = split[1] + '-' + split[-2]
    e, s, r = zfish_track.gui_analyze(video_path, behavior_dir + '/' + fish_trial + '.csv')

In [ ]:
df.diff().iloc[1:]

In [ ]:
import scipy.signal as signal

In [ ]:
b, a = signal.ellip(4, 0.01, 120, 0.125)

In [ ]:
np.array(df['left'])

In [ ]:
signal.filtfilt(b, a, np.array(df['left']), method="gust")

In [ ]:
onset_frames = [2174, 2076, 2170, 2137, 1994, 2957]
for j, csv in enumerate(list_files(behavior_dir)):
    df = pd.read_csv(csv, index_col=0)
    new_df = df.copy()
    for i in new_df:
        new_df[i] = signal.filtfilt(b, a, np.array(df[i]), method="gust")
    new_df.plot()
    plt.axvline(onset_frames[j], c='C2')
    plt.title(csv)

## (WIP) Manually enter the prey capture onset time for each trial

In [ ]:
fish_trials = []
onset_time = []
for i, video_path in enumerate(videos):
    video_name = os.path.split(video_path)[1]
    split = video_name.replace('-', '_').replace('.', '_').split('_')
    fish_trial = split[1] + '-' + split[-2]
    fish_trials.append(fish_trial)
    onset_time.append(onset_frames[i])
pd.DataFrame(onset_time, index=fish_trials, columns=['onset time']).to_csv(f'{processed_dir}/onset_time.csv')